In [28]:
#Install packages
import xarray as xr
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np
import param


#plot using panel
pn.extension()
#Use Dark theme
#pn.config.theme = 'dark'

In [4]:
def param_wrangling():
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #----        Parameter Data.       ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # x parameter data for assessment
    df = pd.read_csv('/glade/campaign/asp/djk2120/PPEn11/csvs/lhc220926.txt',index_col=0)
    # the only dimension here is the 'member' aka file index id [LCH0001-500]
    # convert to xr.ds
    params = xr.Dataset(df)

    # list comprehension no need for empty list
    columns = [params[v].values for v in params.data_vars]

    # iterate over params
    return np.array(columns).T

In [24]:
params.data_vars

Data variables:
    FUN_fracfixers       (member) float64 4kB 0.7783 0.3193 ... 0.2621 0.4221
    KCN                  (member) float64 4kB 0.358 0.5488 ... 0.9123 0.8392
    a_fix                (member) float64 4kB 0.255 0.6302 ... 0.5339 0.1443
    crit_dayl            (member) float64 4kB 0.4012 0.27 ... 0.9974 0.6571
    d_max                (member) float64 4kB 0.7978 0.5361 ... 0.9076 0.4009
    fff                  (member) float64 4kB 0.1301 0.6792 ... 0.9918 0.02798
    froot_leaf           (member) float64 4kB 0.9388 0.6722 ... 0.3635 0.9691
    fstor2tran           (member) float64 4kB 0.3784 0.9566 ... 0.8869 0.1985
    grperc               (member) float64 4kB 0.1965 0.9342 ... 0.7634 0.02519
    jmaxb0               (member) float64 4kB 0.6247 0.6272 ... 0.1925 0.6125
    jmaxb1               (member) float64 4kB 0.3151 0.3516 ... 0.565 0.5094
    kcha                 (member) float64 4kB 0.2852 0.5708 ... 0.7642 0.4513
    kmax                 (member) float64 4kB 0.558

array([[0.77831114, 0.35802577, 0.25502127, ..., 0.39361723, 0.07896155,
        0.68458797],
       [0.31932864, 0.54880317, 0.63019079, ..., 0.04108439, 0.27561916,
        0.87845469],
       [0.87677181, 0.21079367, 0.6621252 , ..., 0.81974643, 0.84014328,
        0.95191497],
       ...,
       [0.18596905, 0.996422  , 0.42220946, ..., 0.24264258, 0.84227481,
        0.7311443 ],
       [0.26212961, 0.9122803 , 0.5339336 , ..., 0.71595008, 0.35301486,
        0.3580522 ],
       [0.42211367, 0.83921159, 0.14434242, ..., 0.21237027, 0.36344002,
        0.1567103 ]])

In [10]:
# import libraries & data pre-processing functions from utils.py
from dash_utils import *

In [2]:
# Request an additional 10 cores of power for processing from the server
client = get_cluster("UCSB0021", cores = 30)
client.cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hmchilders/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.91:37007,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/hmchilders/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [6]:
def subset_and_plot_cluster_panel(param, var):
    '''describe the relationship between the selected variable 
    and parameter(s) between 2005-2010. output a
    cluster plot averaged by year.'''

    # Read in and wrangle user selected parameter cluster
    param_avg  = params[param]

    
    # Read in and wrangle user selected variable cluster
    da_v = read_all_simulations(var)
    # feb. ncar time bug
    da = fix_time(da_v)
    # convert xr.ds to xr.da
    da = da[var]
    # weight gridcell dim by global land area
    da_global = weight_landarea_gridcells(da, landarea)
    # weight time dim by days in month
    da_global_ann = yearly_weighted_average(da_global)
    # take global avg for variable over year dimension
    var_avg = da_global_ann.mean(dim='year')

    data = pd.DataFrame({'x': param_avg, 'y': var_avg})

    # Create scatter plot using hvplot
    scatter_plot = data.hvplot.scatter(
        x='x', y='y', color='#62c900ff', alpha=0.8,
        xlabel=param, ylabel=var, title='2005-2010 Global Average'
    )
    return pn.panel(scatter_plot)



In [16]:
panel_plot = subset_and_plot_cluster_panel('medlynslope', 'QRUNOFF')
panel_plot

HoloViews(Scatter, height=300, sizing_mode='fixed', width=700)

In [7]:
panel_plot = subset_and_plot_cluster_panel('leafcn', 'LNC')
panel_plot

HoloViews(Scatter, height=300, sizing_mode='fixed', width=700)

In [ ]:
def plot_cluster_panel(param, var_avg):
    '''describe the relationship between the selected variable 
    and parameter(s) between 2005-2010. output a
    cluster plot averaged by year.'''

    # Read in and wrangle user selected parameter cluster
    param_avg  = params[param]

    data = pd.DataFrame({'x': param_avg, 'y': var_avg})

    # Create scatter plot using hvplot
    scatter_plot = data.hvplot.scatter(
        x='x', y='y', color='#62c900ff', alpha=0.8,
        xlabel=param, ylabel=var, title='2005-2010 Global Average'
    )
    return pn.panel(scatter_plot)


After Dan's Edits

In [41]:
def plotter(c):
    fig =plt.Figure(figsize=[5,3])
    ax=fig.add_subplot(111)
    x=np.linspace(0,1,20)
    ax.plot(x,c*x)
    ax.set_ylim([0,3])
    ax.set_xlim([0,1])
    return fig

In [39]:
class Selections(param.Parameterized):
    parameter   = param.Selector(objects=[1,2,3])
    variable   = param.Selector(objects=[1,2,3])
    def view(self):
        return plotter(self.parameter)

In [40]:
obj = Selections()
pn.Row(obj.param,obj.view)

TypeError: plotter() missing 1 required positional argument: 'param'